In [2]:
using PyCall

using DrWatson
quickactivate(@__DIR__)

py"""
import pickle
 
def load_pickle(fpath):
    with open(fpath, "rb") as f:
        data = pickle.load(f)
    return data
"""

load_pickle = py"load_pickle"

PyObject <function load_pickle at 0x7f060cabd700>

In [3]:
fname = datadir("exp_pro", "NetSense_network.pkl")
file = load_pickle(fname)

Dict{Any, Any} with 6 entries:
  5 => PyObject <networkx.classes.graph.Graph object at 0x7f05fa670a90>
  4 => PyObject <networkx.classes.graph.Graph object at 0x7f05fa6b5e50>
  6 => PyObject <networkx.classes.graph.Graph object at 0x7f05fa632d90>
  2 => PyObject <networkx.classes.graph.Graph object at 0x7f05fa7325b0>
  3 => PyObject <networkx.classes.graph.Graph object at 0x7f05fa67a1f0>
  1 => PyObject <networkx.classes.graph.Graph object at 0x7f060cd87940>

In [7]:
#works! (shows a lot without semicolon)
file[1].nodes.data();


In [8]:
py"""
import networkx as nx
def get_triads(net):
    A = nx.adjacency_matrix(net).todense() 

    N = A.shape[0]

    triads = []

    for i in range(0,N):
        for j in range(i+1,N):
            if A[i,j] > 0:
                for k in range(j+1,N):
                    if A[i,k] > 0 and A[j,k] > 0:
                        triads.append((i,j,k))
    
    return triads

def get_adjacency_matrix(net):
    return nx.adjacency_matrix(net).todense() 
"""

get_triads = py"get_triads"
get_adjacency_matrix = py"get_adjacency_matrix"

PyObject <function get_adjacency_matrix at 0x7f05fa706c10>

In [9]:
get_triads(file[1])

45-element Vector{Tuple{Int64, Int64, Int64}}:
 (0, 1, 3)
 (0, 3, 4)
 (1, 2, 3)
 (2, 3, 4)
 (3, 8, 15)
 (4, 69, 94)
 (5, 7, 8)
 (5, 7, 17)
 (5, 7, 41)
 (5, 7, 43)
 ⋮
 (12, 53, 80)
 (12, 59, 62)
 (13, 14, 50)
 (15, 19, 20)
 (17, 44, 77)
 (21, 22, 23)
 (35, 36, 83)
 (53, 59, 62)
 (53, 62, 74)

In [10]:
A = get_adjacency_matrix(file[1])

98×98 Matrix{Int64}:
  0  11    0    4   2   0   0   0   0  …  0  0  0   0   0  0   0  0  0  0  0
 11   0   61    8   0   0   0   0   0     0  0  0   0   0  0   0  0  0  0  0
  0  61    0  193  19   0   0   0   0     0  0  0   0   0  0   0  0  0  0  0
  4   8  193    0   5   0   0   0   8     0  0  0   0   0  0   0  0  0  0  0
  2   0   19    5   0   0   0   7   0     0  0  0   0   0  0   0  1  0  0  0
  0   0    0    0   0   0  18  36  49  …  0  0  0   0   0  0   0  0  0  0  0
  0   0    0    0   0  18   0   0   0     0  0  0   0   0  0   0  2  0  0  0
  0   0    0    0   7  36   0   0  81     0  0  0   0   0  0   0  0  0  0  0
  0   0    0    8   0  49   0  81   0     0  0  0   0   0  0   0  0  0  0  0
  0   0    0    0   0   0   0   0   0     0  0  0   0   0  0   0  0  0  0  0
  ⋮                     ⋮              ⋱            ⋮                ⋮     
  0   0    0    0   0   0   0   0   0     0  0  0   0   0  0   0  0  0  0  0
  0   0    0    0   0   0   0   0   0  …  0  0  0   0  8

In [11]:
using LightGraphs

net = Graph(A)

{98, 152} undirected simple Int64 graph

In [84]:
sum(A .> 0)

304

In [72]:
#creating list of attributes
list_of_attributes = Set([k for n in file[1].nodes for k in keys(file[1].nodes[n+1])])

# creating possible attribute values
attribute_values = Dict((attribute, Set() ) for attribute in list_of_attributes)

for sem in 1:6
    for node in file[sem].nodes()
        for attribute in list_of_attributes
            push!(attribute_values[attribute], convert(Int, file[sem].nodes[node + 1][attribute]))
        end
    end
end

attribute_values

#To apply PCA or other analysis on all attributes (i.e. including ethnicity) 
#one would have to create an attribute for each race, that is is_1, is_2 etc. 
#Because this attribute is not ordered. 

Dict{String, Set{Any}} with 12 entries:
  "premaritalsex_" => Set([2, 3, 1])
  "ethnicity_1"    => Set([4, 2, 3, 1])
  "hometown_1"     => Set([2, 3, 1])
  "political_"     => Set([2, 3, 1])
  "abortion_"      => Set([2, 3, 1])
  "homosexual_"    => Set([2, 3, 1])
  "marijuana_"     => Set([2, 3, 1])
  "gender_1"       => Set([2, 1])
  "gaymarriage_"   => Set([2, 3, 1])
  "deathpen_"      => Set([2, 3, 1])
  "euthanasia_"    => Set([2, 3, 1])
  "age_1"          => Set([2, 1])

In [65]:
#creating table of attribute values
attributes = Dict((i, zeros(length(file[i].nodes), length(list_of_attributes))) for i in 1:6)
# for i in 1:6
#     attributes[i] = zeros(length(file[i].nodes), length(list_of_attributes))
# end

for sem in 1:6
    for (i, node) in enumerate(file[sem].nodes())
        for (j, attribute) in enumerate(list_of_attributes)
            attributes[sem][i,j] = file[sem].nodes[node + 1][attribute]
        end
    end
end
attributes

Dict{Int64, Matrix{Float64}} with 6 entries:
  5 => [3.0 2.0 … 1.0 2.0; 3.0 3.0 … 2.0 3.0; … ; 1.0 3.0 … 1.0 3.0; 3.0 3.0 … …
  4 => [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 3.0 … 1.0 1.0; 1.0 1.0 … …
  6 => [3.0 3.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 3.0 3.0 … 2.0 3.0; 1.0 3.0 … …
  2 => [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 3.0 … 1.0 1.0; 2.0 2.0 … …
  3 => [3.0 3.0 … 1.0 2.0; 1.0 1.0 … 1.0 1.0; … ; 3.0 1.0 … 4.0 2.0; 1.0 1.0 … …
  1 => [3.0 3.0 … 2.0 1.0; 2.0 2.0 … 2.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 3.0 3.0 … …

In [70]:
setdiff(list_of_attributes, ["ethnicity_1", "gender_1", "hometown_1", "age_1"])

Set{String} with 8 elements:
  "abortion_"
  "premaritalsex_"
  "homosexual_"
  "marijuana_"
  "euthanasia_"
  "gaymarriage_"
  "deathpen_"
  "political_"

In [77]:

ord_list_of_attributes = setdiff(list_of_attributes, ["ethnicity_1", "gender_1", "hometown_1", "age_1"])
print(ord_list_of_attributes)

#creating table of attribute values
ord_attributes = Dict((i, zeros(length(file[i].nodes), length(ord_list_of_attributes))) for i in 1:6)

for sem in 1:6
    for (i, node) in enumerate(file[sem].nodes())
        for (j, attribute) in enumerate(ord_list_of_attributes)
            ord_attributes[sem][i,j] = file[sem].nodes[node + 1][attribute]
        end
    end
end
# for sem in range(1,7):
#     for i, node in enumerate(data[sem].nodes()):
#         for j, attribute in enumerate(ord_list_of_attributes):
#             ord_attributes[sem][i,j] = data[sem].nodes[node][attribute]
ord_attributes

Set(["abortion_", "premaritalsex_", "homosexual_", "marijuana_", "euthanasia_", "gaymarriage_", "deathpen_", "political_"])

Dict{Int64, Matrix{Float64}} with 6 entries:
  5 => [3.0 2.0 … 1.0 2.0; 3.0 3.0 … 2.0 3.0; … ; 1.0 3.0 … 1.0 3.0; 3.0 3.0 … …
  4 => [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 3.0 … 2.0 1.0; 1.0 1.0 … …
  6 => [3.0 3.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 3.0 3.0 … 3.0 3.0; 1.0 3.0 … …
  2 => [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 3.0 … 3.0 1.0; 2.0 2.0 … …
  3 => [3.0 3.0 … 1.0 2.0; 1.0 1.0 … 1.0 1.0; … ; 3.0 1.0 … 1.0 2.0; 1.0 1.0 … …
  1 => [3.0 3.0 … 3.0 1.0; 2.0 2.0 … 2.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 3.0 3.0 … …